In [ ]:
import cantools
from pprint import pprint
import pandas as pd

In [ ]:
db = cantools.database.load_file('testing_data/20240129 Gen5 CAN DB.dbc')

In [ ]:
print(db)

In [ ]:
import cantools
import re
import pandas as pd
from pprint import pprint

def parse_can_data_bytes(data_bytes_str):
    # Split by one or more spaces, convert to bytes
    return bytes(int(byte) for byte in data_bytes_str.split() if byte)

def parse_can_message_to_df(line, db):
    """
    Parse a CAN message line and decode its contents using a cantools database.
    Export the result as a Pandas DataFrame.
    Args:
        line (str): CAN message line to parse.
        db (cantools.database.Database): Loaded CAN database.
    Returns:
        pd.DataFrame: Decoded message as a DataFrame.
    """
    # Regular expression to parse the line
    # pattern = r'\s*(\d+)\s+(\w+)\s+(\d+)\s+([0-9\s]+)\s+(\d+\.\d+)\s+([RX])'
    #TODO: seems like the DBC file is not correct, or the data is not. DBC missing 2048, etc.
    pattern = r'\s*(\d+)\s+(\w+)(?:\s+X)?\s+(\d+)\s+([0-9\s]+)\s+(\d+\.\d+)\s+([RX])'
    match = re.match(pattern, line)
    if not match:
        raise ValueError("Invalid CAN message line format")

    # Extract components
    index, can_id, dlc, data_bytes, timestamp, direction = match.groups()
    can_id_int = int(can_id)

    # Get message from database
    try:
        message = db.get_message_by_frame_id(can_id_int)
    except KeyError:
        return pd.DataFrame([{
            "Error": f"No message found for CAN ID {can_id_int}",
            "Raw Data": {
                "Index": index,
                "CAN ID": can_id,
                "DLC": dlc,
                "Data Bytes": data_bytes,
                "Timestamp": timestamp,
                "Direction": direction
            }
        }])

    # Prepare data bytes for decoding
    data_list = parse_can_data_bytes(data_bytes)

    # Decode message
    try:
        decoded_data = message.decode(data_list)
    except Exception as e:
        return pd.DataFrame([{
            "Error": f"Decoding error: {str(e)}",
            "Raw Data": {
                "Index": index,
                "CAN ID": can_id,
                "DLC": dlc,
                "Data Bytes": data_bytes,
                "Timestamp": timestamp,
                "Direction": direction
            }
        }])

    # Create a list to store signal data
    signal_data = []
    for signal_name, value in decoded_data.items():
        # Find the signal by name
        signal = next((sig for sig in message.signals if sig.name == signal_name), None)
        signal_data.append({
            "Signal Name": signal_name,
            "Value": value,
            "Description": signal.comment if signal else "No description available",
            "Unit": signal.unit if signal else "N/A",
            "Message Name": message.name,
            "Raw CAN ID": can_id,
            "Timestamp": timestamp,
            "Direction": direction
        })

    # Convert signal data to a DataFrame
    return pd.DataFrame(signal_data)

In [ ]:
# Example usage
line = "0         514        8 120   0   6   0   0   0   0   0      19.439960 R"
db = cantools.database.load_file('testing_data/20240129 Gen5 CAN DB.dbc')
df = parse_can_message_to_df(line, db)

# Print the DataFrame
pprint(df)

In [ ]:
df.head(2)

canBus,sensorId=A0100,system=motor rpm=3500,torque=10 1636729543000000000
canBus,signalName=df['Signal Name'][i],description=df['Description'][i],messageName=df['Message Name'], RawCAN=df['Raw CAN ID'][i] sensorReading=df['Value'][i],unit=df['Unit'][i],relativeTime=df['Timestamp'][i] append time at the end

In [ ]:
from datetime import datetime

# Assuming df is your pandas DataFrame
for i in range(len(df)):
    now = datetime.now()
    # Generate Unix timestamp with nanoseconds
    unix_time_ns = int(now.timestamp() * 1e9)
    formatted_string = (
        f"canBus,"
        f"signalName={df['Signal Name'][i]},"
        f"description={df['Description'][i]},"
        f"messageName={df['Message Name'][i]},"
        f"RawCAN={df['Raw CAN ID'][i]} "
        f"sensorReading={df['Value'][i]},"
        f"unit={df['Unit'][i]},"
        f"relativeTime={df['Timestamp'][i]} "
        f"time={unix_time_ns}"
    )
    print(formatted_string)  # Replace with any action like saving or logging

In [ ]:
import multiprocessing
from datetime import datetime
import pandas as pd

def process_line(line, db):
    """
    Process a single line, parse CAN message, and generate formatted strings.
    """
    try:
        df = parse_can_message_to_df(line, db)
        formatted_strings = []
        for j in range(len(df)):
            now = datetime.now()
            # Generate Unix timestamp with nanoseconds
            unix_time_ns = int(now.timestamp() * 1e9)
            formatted_string = (
                f"canBus,"
                f"signalName={df['Signal Name'][j]},"
                f"description={df['Description'][j]},"
                f"messageName={df['Message Name'][j]},"
                f"RawCAN={df['Raw CAN ID'][j]} "
                f"sensorReading={df['Value'][j]},"
                f"unit={df['Unit'][j]},"
                f"relativeTime={df['Timestamp'][j]} "
                f"time={unix_time_ns}"
            )
            formatted_strings.append(formatted_string)
        return formatted_strings
    except Exception as e:
        print(f"Error processing line: {e}")
        return []

def parallel_process_lines(lines, db):
    """
    Process lines in parallel and collect results.
    """
    with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        # Use starmap to pass both the line and db to the process_line function
        results = pool.starmap(process_line, [(line, db) for line in lines])
    # Flatten the list of lists (results) into a single list of formatted strings
    output = [item for sublist in results for item in sublist]
    return output




In [ ]:
output = []
for i in range(len(lines)):
    try:
        df = parse_can_message_to_df(lines[i], db)
        for j in range(len(df)):
            now = datetime.now()
            # Generate Unix timestamp with nanoseconds
            unix_time_ns = int(now.timestamp() * 1e9)
            formatted_string = (
                f"canBus,"
                f"signalName={df['Signal Name'][j]},"
                f"description={df['Description'][j]},"
                f"messageName={df['Message Name'][j]},"
                f"RawCAN={df['Raw CAN ID'][j]} "
                f"sensorReading={df['Value'][j]},"
                f"unit={df['Unit'][j]},"
                f"relativeTime={df['Timestamp'][j]} "
                f"time={unix_time_ns}"
            )
        # print(formatted_string)  # Replace with any action like saving or logging
    except:
        print('error')
        pass

    # append db into a db
    output.append(df)

In [ ]:
# save db into csv
output.to_csv('testing_data/CanTraceJuly11_parsed.csv')